In [ ]:
import flopy
import os
import geopandas as gpd
import basic
import contextily as ctx
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import pathlib
import geopandas as gpd
import conda_scripts.plot_help as ph
import basic
import conda_scripts.arich_functions as af
import cartopy.crs as ccrs

import pathlib 

In [ ]:
import postprocess

In [ ]:
import basic

m = basic.load_model()

In [ ]:
ISWRPQAQ, ISWRPRGF, ISWRPSTG, ISWRPSTR, ISWRPQM = postprocess.SWR(m, datestart, max_reach = 116, remove_ponds = True  )

In [ ]:
ax = ISWRPQAQ.plot(figsize = (15, 10), subplots = True, title = 'Pond Water Levels', color = 'b')

wl.plot(ax = ax, subplots = True, color = 'k')

In [ ]:
postprocess.plot_rds_stage(m, datestart, out_folder, numdays = 109)

In [ ]:
ISWRPQAQ, wl = postprocess.plot_ponds(m, datestart)

In [ ]:
import pond_inflows

pond_inflows.run('June2012')

In [ ]:
import geopandas as gpd
import numpy as np

In [ ]:
pp = gpd.read_file("ponds/ponds.geojson")

In [ ]:
import pandas as pd
import pathlib

In [ ]:
def isnumber(x):
    try:
        float(x)
        return x
    except:
        return np.nan

In [ ]:
def load_pond_wl(pond = 1):
    p =r"file:///T:\arich\Russian_River\MirabelWohler_2022\Waterlevel_Data\MWs_Caissons%20-%20AvailableDailyAverages\DailyData\InfiltrationPonds"
    
    path = pathlib.Path(p).joinpath(f"Pond{pond}WaterLevel.csv")
    
    tab = pd.read_csv(path).set_index('DateTime').loc[:,['Value']]
    
    tab.index = pd.to_datetime(tab.index)
    
    tab.loc[:,'Value'] = tab.loc[:,'Value'].apply(isnumber)
    tab = tab.astype({'Value':np.float64})
    
    tab.loc[tab.loc[:,'Value'].abs()>10] = np.nan
    tab.loc[tab.loc[:,'Value']<0] = 0.0
    
    return tab

pond = load_pond_wl(4)
    

In [ ]:
pond.head().values

In [ ]:
pond.plot()

In [ ]:
m = basic.load_model()

info, swr_info, sfr_info, riv_keys_info = basic.load_params('June2012')

datestart = info['start_date']
numdays = info['numdays']

name = info['name']

out_folder = basic.out_folder('June2012')

basic
print(datestart)
print(out_folder)

In [ ]:
info

In [ ]:
grid = gpd.read_file("GIS/grid.shp")

In [ ]:
ponds = gpd.read_file(r"C:\Users\arich\Documents\ArcGIS\Projects\MirabelWohler_MODFLOW\MirabelWohler_MODFLOW.gdb", layer = 'ponds')

In [ ]:
pond_grid = gpd.sjoin( grid, ponds).drop_duplicates('node')
ax = pond_grid.plot('name', edgecolor = 'k')

ponds.plot('name',ax = ax, edgecolor = 'w',facecolor = "None", hatch = '\\\\')

_gdf = pond_grid.dissolve('name').reset_index()
_gdf = _gdf.set_geometry(_gdf.geometry.representative_point())

_gdf.plot(ax = ax)

In [ ]:
fig, ax = basic.basic_map()
basic.set_bounds_to_shape(ax, pond_grid.buffer(5000))
pond_grid.sort_values('name').plot('name', ax = ax, legend = False, edgecolor = 'k')
ax.set_title('Mirabel-Wohler Infiltration Ponds')

_gdf = pond_grid.dissolve('name').reset_index()
_gdf = _gdf.set_geometry(_gdf.geometry.representative_point())

ph.label_points(ax, _gdf,
                'name', basin_name = None, fmt = 's', text_color = 'y')
plt.savefig(os.path.join(out_folder, 'pondloc.png'), dpi = 250)

In [ ]:
max_reach = 116

pond_grid.loc[:,'rno'] = np.arange(max_reach+1, max_reach + pond_grid.shape[0]+1)

pond_grid.loc[:,'irgnum'] = pond_grid.loc[:,'name'].replace({'One':1, 'Two':2, 'Three':3, 'Four':4})+max_reach

pond_grid = pond_grid.sort_values(['irgnum','rno'])

top = m.dis.top

pond_grid.loc[:,'top'] = top[pond_grid.loc[:,'row']-1, pond_grid.loc[:,'column']-1]

In [ ]:
def assign_inflow(df_pond):
    
    dfall = pd.DataFrame()
    
    for _, pond in df_pond.groupby('name'):
        pond.loc[:,'inflow'] = False
        
        pond.loc[pond.loc[:,'top'].idxmin(),'inflow'] = True
        
        pond.loc[:, 'mean_elev'] = pond.loc[:, 'top'].mean()
        
        dfall = dfall.append(pond)
        
    return dfall

pond_grid = assign_inflow(pond_grid)

In [ ]:

fig, ax = basic.basic_map(maptype = None)
basic.set_bounds_to_shape(ax, pond_grid.buffer(1000))
pond_grid.sort_values('name').plot('name', ax = ax, legend = False, edgecolor = 'k')
ax.set_title('Mirabel-Wohler Infiltration Ponds Inflow Locations')


ax = pond_grid.plot('top', ax = ax, legend = True, legend_kwds = {'shrink': .2})

pond_grid.set_geometry(pond_grid.geometry.centroid).loc[pond_grid.loc[:,'inflow']].plot(ax = ax, color = 'k')

pond_grid.plot(ax = ax, facecolor = "None")

ph.label_points(ax, pond_grid.set_geometry(pond_grid.geometry.centroid).loc[pond_grid.loc[:,'inflow']],
                'name', basin_name = None, fmt = 's', text_color = 'k')

af.add_basemaps(ax, maptype = "ctx.Esri.NatGeoWorldMap")

plt.savefig(os.path.join(out_folder, 'pondloc_inflow.png'), dpi = 250)

In [ ]:
tarray = top.array

In [ ]:
pd.DataFrame(pond_grid).loc[:,['name', 'mean_elev','top']].hist(by = 'name')

In [ ]:
tarray[pond_grid.loc[:,'row']-1, pond_grid.loc[:,'column']-1] =  pond_grid.loc[:,'mean_elev']

In [ ]:
np.savetxt('RR_2022/inputs/lay_1_top.csv', tarray, fmt = '%.3f', delimiter = ',')

In [ ]:
pond_grid.to_file('ponds/ponds.geojson', )

In [ ]:
print(pond_grid.rno.max())

In [ ]:
def check_lastline(file):
    with open(file,'r') as dr:
        d = dr.readlines()
    
    if '\n' in d[-1]:
        pass
    else:
        with open(file, 'a') as wr:
            wr.write('\n')

# Function to find adjacent
# elements in List
def findAdjacentElements(index, test_list):
    res = []
    for idx, ele in enumerate(test_list):
    # for idx, ele in zip(index, test_list):
  
        # Checking for all cases to append
        if idx == 0:
            res.append((1, test_list[idx + 1]-1))
        elif idx == (len(test_list) -1):
            res.append((1, test_list[idx - 1]-1))
            print(idx)
        else:
            res.append((2, index[idx]-1, test_list[idx]))
    return dict(zip(index, res))

def find_route(sfr, IROUTETYPE  = 4, rchlen = 100 ):

    
    sfr = sfr.sort_values('rno')
    # min_r = sfr.rno.min() - 1 # offset to get reach 1 
    sfr.loc[:,'rno_og'] = sfr.loc[:,'rno'].copy()
    # sfr.loc[:,'rno'] = np.arange(1, sfr.shape[0]+1)
    sfr.loc[:,'outreach'] = sfr.loc[:,'rno'].shift(-1, fill_value=-999)
    # sfr.loc[:,'rno'] =  sfr.loc[:,'rno'] - min_r
    # sfr.loc[:,'outreach'] =  sfr.loc[:,'outreach'] - min_r
    sfr.loc[:,'ieqn'] = IROUTETYPE 
    # sfr.loc[:,'irgnum'] = sfr.loc[:,'rno']
    sfr.loc[:,'krch'] = 1 # layer 1
    sfr.loc[:,'IRCH'] = sfr.loc[:,'row']
    sfr.loc[:,'JRCH'] = sfr.loc[:,'column']
    sfr.loc[:,'rchlen'] = rchlen
    # reach location data
    #IREACH IEQN IRGNUM KRCH   IRCH JRCH   RLEN
       # 1    4     01    1     161  62  304.8
       # 2    4     02    1     162  62  304.8
       # 3    4     03    1     163  61  304.8
       # 4    4     04    1     164  61  304.8
       # 5    4     05    1     165  61  304.8
    reach_loc = sfr.loc[:,['rno','ieqn', 'irgnum', 'krch', 'IRCH', 'JRCH', 'rchlen' ]]

    rout = findAdjacentElements(sfr.loc[:,'rno'].values, sfr.loc[:,'outreach'].values, )
        
    return sfr, reach_loc, rout

def write_reach_connect(rout):
    check_lastline('RR_2022/inputs/rout.txt')
    with open('RR_2022/inputs/rout.txt','a') as r:
        for key, value in rout.items():
            v = '    '.join([str(v) for v in value])
            r.write(f"{key}   {v}\n")
            
def write_reach_loc_dat(sfr_filt):
    check_lastline('RR_2022/inputs/reach_loc.txt')
    sfr_filt.to_csv('RR_2022/inputs/reach_loc.txt', sep = ' ', index = False, header = False, mode = 'a')

def write_stress_period(sfr_filt):
    '''
      # DATASET 5 - STRESS PERIOD 1
    # ITMP IRDBND IRDRAI IRDEVP IRDLIN IRDGEO IRDSTR IRDSTG IPTFLG [IRDAUX]
     1     5    0     0       1     5      1     5      1      # 5  
    
    '''

    n = sfr_filt.shape[0]
    with open("RR_2022/inputs/stress_period_data.txt",'w') as sp:
        sp.write("# DATASET 5 - STRESS PERIOD\n")
        sp.write( "# ITMP IRDBND IRDRAI IRDEVP IRDLIN IRDGEO IRDSTR IRDSTG IPTFLG [IRDAUX]\n")
        sp.write(f"     1     {n}    0     0       1     {n}      1     {n}      1      # 5")
        
def write_dataset_6(sfr_filt):
    check_lastline('RR_2022/inputs/dataset6.txt')
    with open('RR_2022/inputs/dataset6.txt','a') as r:
        for i in sfr_filt.loc[:,'rno']:
            r.write(f"{i}  1\n")
    
def write_dataset_evap(sfr_filt,evap =0.000):
    sfr_filt.loc[:,'evap'] = evap
    # sfr_filt.loc[:,['rno','evap']]
    sfr_filt.loc[:,['rno','evap']].rename(columns = {'rno':'#rno'}).to_csv('RR_2022/inputs/evap.tab', sep = ' ', index = False)
    
def write_dataset_rain(sfr_filt,rain =0.000):
    sfr_filt.loc[:,'rain'] = rain
    # sfr_filt.loc[:,['rno','evap']]
    sfr_filt.loc[:,['rno','rain']].rename(columns = {'rno':'#rno'}).to_csv('RR_2022/inputs/rain.tab', sep = ' ', index = False)
    
def write_dataset_10(sfr_filt, shift = 0):
    # DATASET 10 - GEOMETRY ASSIGNMENT DATA
    sfr_filt.loc[:,'IGMODRCH'] = sfr_filt.loc[:,'rno']
    sfr_filt.loc[:,'IGEONUMR'] = sfr_filt.loc[:,'rno']
    sfr_filt.loc[:,'GZSHIFT'] = shift
    #IGMODRCH IGEONUMR GZSHIFT 
    check_lastline('RR_2022/inputs/dataset_10.txt')
    sfr_filt.loc[:,['IGMODRCH','IGEONUMR','GZSHIFT' ]].rename(columns = {"IGMODRCH":"#IGMODRCH"}).to_csv('RR_2022/inputs/dataset_10.txt', sep = ' ', index = False, header = False, mode = 'a')
    

    
def write_dataset_14a(sfr_filt, m, shift = 5):
    bot = m.dis.top
    sfr_filt.loc[:,'top'] = bot[sfr_filt.loc[:,'i'], sfr_filt.loc[:,'j']] + shift
    
    # print(sfr_filt.loc[:,'top'])
    check_lastline('RR_2022/inputs/dataset_14a.txt')
    sfr_filt.loc[:,['rno', 'top']].to_csv('RR_2022/inputs/dataset_14a.txt', sep = ' ', index = False, mode = 'a')
        
        
def write_start_stage(sfr_filt):
    '''
    uses ending heads of last model run to create new starting heads
    '''    
    check_lastline('RR_2022/inputs/start_stage.tab')
    sfr_filt.loc[:,'top'] = 0
    sfr_filt.loc[:,['rno','top']].to_csv('RR_2022/inputs/start_stage.tab', sep = '\t', index = False, header = False, mode = 'a')
        
def write_geo(sfr_filt,IGEOTYPE = 5, IGCNDOP = 1, GMANNING = 0.025, GLK = 0.001, GETEXTD = 0.1):
    "IGEONUM IGEOTYPE IGCNDOP GMANNING [NGEOPTS] [GWIDTH] [GBELEV] [GSSLOPE] [GCND] [GLK] [GCNDLN] [GETEXTD]"
    
    df = sfr_filt.copy()
    df.loc[:,'IGEONUM'] = df.loc[:,'rno']
    df.loc[:,'IGEOTYPE'] = IGEOTYPE
    df.loc[:,'IGCNDOP'] = IGCNDOP
    df.loc[:,'GMANNING'] = GMANNING
    df.loc[:,'GLK'] = GLK
    df.loc[:,'GETEXTD'] = GETEXTD
    df.loc[:,[ "IGEONUM","IGEOTYPE","IGCNDOP","GMANNING", "GLK", "GETEXTD"]].to_csv('RR_2022/inputs/dataset11a_with_geo.txt', sep = '\t', index = False, header = False, mode = 'a')
    
    
    

for name, pi in pond_grid.groupby('irgnum'):
    sfr_filt, reach_loc, rout = find_route(pi, IROUTETYPE = 1)
#     write_reach_connect(rout)
    
#     write_reach_loc_dat(reach_loc)
    
#     write_dataset_10(sfr_filt, shift = 0)
    
#     write_start_stage(sfr_filt)
    
#     write_dataset_6(reach_loc)
    
    # write_geo(sfr_filt)
    
print(f"structures:\n{pond_grid.drop_duplicates('name').rno.values}")
# reach_min = 578
#     # bottom, n = 10, reach_min = 578, IROUTETYPE  = 4,    
# sfr_filt, reach_loc, rout = find_route(sfr, 
#                                        bottom, 
#                                        n = None, 
#                                        reach_min = reach_min,
#                                        IROUTETYPE = IROUTETYPE, 
#                                        use_average_reach_len=use_average_reach_len)

# print(sfr_filt.shape)

# write_reach_connect(rout)

# write_reach_loc_dat(reach_loc)

# write_stress_period(sfr_filt)

# write_dataset_6(sfr_filt)

# write_dataset_evap(sfr_filt,evap)
# write_dataset_rain(sfr_filt,rain)

# write_dataset_10(sfr_filt)


# write_dataset_14a(sfr_filt, m)




    
# plot_swr(sfr_filt)

# reach_loc

In [ ]:
def load_phist(year = 2020):
    p = pathlib.Path(r"S:\Ops\RiverReport\Production_and_Demand_Report_PHIST01.xlsm")
    
    sheet = f"Data_{year}"
    
    tab = pd.read_excel(p, sheet_name=sheet, header = [3,4,5,6,7,8], skiprows=[9,10,11,12], index_col=[0])
    
    return tab

def isnumber(x):
    try:
        float(x)
        return x
    except:
        return np.nan

def load_pond(datestart):
    
    year = pd.to_datetime(datestart).year
    
    df = load_phist(year)
    
    df = df.loc[:,'River Diversion'].iloc[:,:3].droplevel([0,1,3,4], axis = 1)
    

    df = df.applymap(isnumber).fillna(0)
    
    # df = df.astype({'Pump 1':np.float64})
    print(df.dtypes)
    df.loc[:,'Pump 1'] = df.loc[:,'Pump 1'] *18000*60/(7.48*60*60*24)
    df.loc[:,'Pump 2'] = df.loc[:,'Pump 2'] *9000*60/(7.48*60*60*24)
    df.loc[:,'Pump 3'] = df.loc[:,'Pump 3'] *18000*60/(7.48*60*60*24)
    
    #get water levels
    wl = pd.DataFrame()
    for pond in [1,2,3,4]:
        p = pathlib.Path(r"T:\arich\Russian_River\MirabelWohler_2022\Waterlevel_Data\MWs_Caissons - AvailableDailyAverages\DailyData\InfiltrationPonds")

        p = p.joinpath(f"Pond{pond}WaterLevel.csv")
        
        c = pd.read_csv(p)
        c.loc[:,'Value'] = c.loc[:,'Value'].apply(isnumber)
        c = c.astype({'Value':np.float64})
        c = c.set_index(pd.to_datetime(c.loc[:,'DateTime']))
        c = c.loc[:,['Value']].resample('1D').mean()
        c = c.rename(columns = {"Value":f"Pond{pond}"})
        wl = wl.join(c, how = 'outer')

    wl[wl.abs()>100] = np.nan
    wl[wl<0] = 0.
    
    wl = wl.interpolate('spline',  order=2)
    
    return df, wl
    
def get_period(df, start_date, numdays):
    
    df = df.sum(axis = 1).to_frame("Value")
    
    df = df.loc[start_date:,:].iloc[:numdays]
    
    df.loc[:,'Value'] = df.loc[:,'Value'].interpolate()
    
    assert df.loc[:,'Value'].isnull().sum() ==0 , 'has nans'
    
    return df
    
df, wl = load_pond(datestart)

df_cur = get_period(df, datestart, numdays)
df_cur.head()

In [ ]:
from cycler import cycler
import numpy as np
import matplotlib.pyplot as plt

default_cycler = (cycler(color=['r', 'g', 'b', 'y', 'r', 'g', 'b', 'y']) +
                  cycler(linestyle=['-', '-', '-', '-', '--', '--','--','--']))

plt.rc('lines', linewidth=4)
plt.rc('axes', prop_cycle=default_cycler)

In [ ]:
wl.plot(figsize = (15,10),subplots = True, title = 'Pond Water Levels')

In [ ]:
inflow_fraction = {'One':0, 'Two':.5, 'Three':.5, "Four":0}

fraction = pd.DataFrame.from_dict(inflow_fraction, orient  = 'index', columns =[ 'fraction'])

fraction

In [ ]:
ax = df_cur.plot(ylabel = 'feet$^3$/s', figsize = (7,7))
df_cur_roll = df_cur.rolling(5, min_periods =0).mean(center = False)

df_cur_roll.rename(columns = {'Value':'Value, Rolled'}).plot.area(ax = ax)
ax.axvline(pd.to_datetime(datestart) + pd.to_timedelta(22, unit = 'D'), c = 'k', ls = '--')
ax.set_title('Pond Inflows, Split Between 2 and 3')

plt.savefig(os.path.join(out_folder, 'pondQ.png'), dpi = 250)

In [ ]:
# temp for bug checkking
df_cur_roll.iloc[3:30,0] = np.arange(3,30)

ax = df_cur_roll.plot()
ax.axvline(pd.to_datetime(datestart) + pd.to_timedelta(22, unit = 'D'), c = 'k', ls = '--')

In [ ]:
cnt = 0
for ind, d in df_cur_roll.iterrows():
    
    pinf = pond_grid.query("inflow==True").set_index('rno')

    fraction = pd.DataFrame.from_dict(inflow_fraction, orient  = 'index', columns =[ 'fraction'])

    pinf = pd.merge(pinf, fraction, left_on = 'name', right_index = True)
    pinf = pinf.loc[:,['fraction']]

    pinf = pinf*d['Value']
    

    print(d['Value'])
    print(ind)
    name = f"RR_2022/ref/pond/day{cnt}.dat"
    print(name)
    with open(name,'w') as out:
        
        pinf.to_csv(name, header = False)
        # out.write(f.format(d['Value'], ind.strftime("%y %b %d")))
   
    cnt = cnt+1
    


In [ ]:
df.plot.area(stacked = True, ylabel = 'feet$^3$ per day')